<a href="https://colab.research.google.com/github/gageswanston/BNFO301_Swanston_Gage/blob/main/BNFO301_2024_Lab5_1_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>BNFO301 Lab Assignment 5.1: Consensus Sequences</h1>

A consensus sequence represents the most frequent nucleotide at each position in an alignment of two or more sequences. You can think of this as finding the most common base in a position within a multiple sequence alignment. Consensus sequences can be useful for identifying and visualizing motifs in a DNA or amino acid sequence. You have been provided a FASTA file of aligned sequences. The goal of this assignment is to write a python script to read in the sequences from the file and generate the consensus sequence


Grading: problem 1 = 4 points; problem 2 = 6 points. 1 point extra-credit.

Timeline: You will have two class periods to complete the lab.  Part 1 is given this week.  You recieve a second part next week.    Your responses to problems should be uploaded to **github**.  Your reponse to the extra-credit should be uploaded to **canvas**.

**Helpful Resources:**

If you are unfamiliar with consensus sequences or would like to review, this video explains the concept: https://www.youtube.com/watch?v=4HYJILahPw4 between 2:20 and 4:20. It may also be helpful to first generate the consensus sequence by hand, so that you can check the output from your script.

**Reminder: You may work with other students in the class, but you are not allowed to copy code from others or any source.  If you copy anything from any source, you must describe the source.  Failure to comply is a violation of the class policy.**

### Setup
Load the Sequence Data File.  Please run this block without changing the code.

This file is in fasta format and contains an alignment of DNA sequences, including gaps

you can view the file by clicking on this link: https://raw.githubusercontent.com/boydvcu/BNFO301_2023/main/Nucl_alignment.fa

In [1]:
import os.path
# Load the genbank file
DATA_FILE_GITHUB = "https://raw.githubusercontent.com/boydvcu/BNFO301_2023/main/Nucl_alignment.fa"
DEFAULT_FILE_NAME = 'Nucl_alignment.fa'

fileName = DEFAULT_FILE_NAME
#Does the file exists locally, if not get it from the github
if not os.path.exists(fileName):
  #Load the file from Github to the local folder
  !wget --no-check-certificate --content-disposition $DATA_FILE_GITHUB



--2024-02-15 17:40:12--  https://raw.githubusercontent.com/boydvcu/BNFO301_2023/main/Nucl_alignment.fa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 302 [text/plain]
Saving to: ‘Nucl_alignment.fa’

Nucl_alignment.fa   100%[===================>]     302  --.-KB/s    in 0s      

2024-02-15 17:40:12 (6.45 MB/s) - ‘Nucl_alignment.fa’ saved [302/302]



###problem-1.  Create a definition to read the FASTA formated file and add sequences into a list.  Create a second list that contains your headers (with the ">" removed).  Your function should return both lists, not just print each list.  You have been provided a fucntion to print each lise, so that you can evaluate your lists.


Hint, you will need to do the following:
* We need a function to load the sequence file
* We will load it in two lists of sequences and header names
* Remember to strip ">" and "\n" characters


Your output should look like this:

Human       GGAGAGGCTCGGAGCCGGGCCCGGACCCCGGCGATTGCCGCCCGCTTCTCT

Gorilla     GGAGAGGCTCGGAGCCGGGCCCGGACCCCGGCGATTGCCGCCCGCTTCTCT

Dolphin     GAGGCTC---GGAGCCGGACCTGGACCCCTGCGATAGCCGTCTG-CTCCCG

Rat         GGAGCAACTAGGAACCCGAACCAGAGCCCGGCGAGCGCAGCCTGCAGCTCC

Mouse       GAGGCGCCTAGGAACCCGAGCCGGAGCTCAGCGAGCGCAGCCTGCAGCTCC


In [12]:
from os import linesep
#-----------------------------------------
#1.1. create a function to read the fasta formated file and return both a list of sequences and a list of sequence identifiers
#-----------------------------------------
def ReadFile(fh):
  seq=[]
  ident=[]
  with open (fh, 'r') as infile:
    for line in infile:
      if (line[0]=='>'):
        ident.append(line.strip('> \n'))
      else:
        seq.append(line.strip())
  return ident,seq


# -----------------------------
#1.2 you have been provided with the function below to print sequences and sequence_names in an easily readable format
# ------------------------------
def printInfo(seqIds, seqs):
    for seqId, seq in zip(seqIds, seqs):
        print("{: <12} {: <20}".format(seqId, seq))

# -----------------------------
#1.3 remember to call your functions
# -----------------------------
tup=ReadFile(fileName)
printInfo(tup[0],tup[1])


Human        GGAGAGGCTCGGAGCCGGGCCCGGACCCCGGCGATTGCCGCCCGCTTCTCT
Gorilla      GGAGAGGCTCGGAGCCGGGCCCGGACCCCGGCGATTGCCGCCCGCTTCTCT
Dolphin      GAGGCTC---GGAGCCGGACCTGGACCCCTGCGATAGCCGTCTG-CTCCCG
Rat          GGAGCAACTAGGAACCCGAACCAGAGCCCGGCGAGCGCAGCCTGCAGCTCC
Mouse        GAGGCGCCTAGGAACCCGAGCCGGAGCTCAGCGAGCGCAGCCTGCAGCTCC


###problem-2. In this problem you will build a defition that uses nested loops to format the data in such a way that you can determine the most common base.  Multiple bioinformatic programs handle data in this way.  Remember to return your data as strings that represent bases in a column.  You have been given code to print the first three columns of the matrix.  This allows you to visualize your matrix and check it for accuracy.  Functions len and range will be useful in completing this problem.


For example, lets take the input for the first three bases in the alignment given:

Human       GGA

Gorilla     GGA

Dolphin     GAG

Rat         GGA

Mouse       GAG

You should return data that looks like this:  [['G', 'G', 'G', 'G', 'G'], ['G', 'G', 'A', 'G', 'A'], ['A', 'A', 'G', 'A', 'G']]

In [35]:
# -----------------------------
# Convert sequence format for easy computation
# ------------------------------
def maniputlateSeqs(seqs):
    # Note lengths of the aligned sequences are equal in length
    #first, initialize a list to store your columns
    columndata=[]

    # Create a list of lists using nested loops
    # Example:
    #        ATGCA
    #        ATGAA
    #        TCGAT
    #             Bases at index 0   Bases at index 1 ...
    # positions = [["A", "A", "T"],  ["T", "T", "C"], ...]]
    # this process is best done with a for loop
    # the append function will be most useful
    #do not forget to return the data
    for i in range(len(seqs[0])):
      Mtx = []
      for seq in seqs:
        Mtx.append(seq[i])
      columndata.append(Mtx)
    #print(columndata)
    print(columndata[0:3])


#print the first 3 rows of the matrix
#print(columndata[0:3])

maniputlateSeqs(ReadFile(fileName)[1])





[['G', 'G', 'G', 'G', 'G'], ['G', 'G', 'A', 'G', 'A'], ['A', 'A', 'G', 'A', 'G'], ['G', 'G', 'G', 'G', 'G'], ['A', 'A', 'C', 'C', 'C'], ['G', 'G', 'T', 'A', 'G'], ['G', 'G', 'C', 'A', 'C'], ['C', 'C', '-', 'C', 'C'], ['T', 'T', '-', 'T', 'T'], ['C', 'C', '-', 'A', 'A'], ['G', 'G', 'G', 'G', 'G'], ['G', 'G', 'G', 'G', 'G'], ['A', 'A', 'A', 'A', 'A'], ['G', 'G', 'G', 'A', 'A'], ['C', 'C', 'C', 'C', 'C'], ['C', 'C', 'C', 'C', 'C'], ['G', 'G', 'G', 'C', 'C'], ['G', 'G', 'G', 'G', 'G'], ['G', 'G', 'A', 'A', 'A'], ['C', 'C', 'C', 'A', 'G'], ['C', 'C', 'C', 'C', 'C'], ['C', 'C', 'T', 'C', 'C'], ['G', 'G', 'G', 'A', 'G'], ['G', 'G', 'G', 'G', 'G'], ['A', 'A', 'A', 'A', 'A'], ['C', 'C', 'C', 'G', 'G'], ['C', 'C', 'C', 'C', 'C'], ['C', 'C', 'C', 'C', 'T'], ['C', 'C', 'C', 'C', 'C'], ['G', 'G', 'T', 'G', 'A'], ['G', 'G', 'G', 'G', 'G'], ['C', 'C', 'C', 'C', 'C'], ['G', 'G', 'G', 'G', 'G'], ['A', 'A', 'A', 'A', 'A'], ['T', 'T', 'T', 'G', 'G'], ['T', 'T', 'A', 'C', 'C'], ['G', 'G', 'G', 'G', 'G'], 

###Extra-credit  
One way aligned sequences are often visualized is as a sequence logo. For an additional point, go to https://weblogo.berkeley.edu/logo.cgi and upload the provided FASTA file. Save the image and upload to canvas.  